In [15]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.utils.data import Dataset, DataLoader
from time import perf_counter 
import math
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
%matplotlib inline
 
def to_torch(state):
    state = torch.from_numpy(state).float()
    return state
 
class RNN(nn.Module):
    def __init__(self, p, q, output_size):
        super(RNN, self).__init__()
        self.p = p
        self.q = q

        self.hidden_dim = q
        self.feature_size = p
        self.output_size = output_size
        self.rnn = nn.RNN(self.feature_size, self.hidden_dim)
        self.hidden2output = nn.Linear(self.hidden_dim, output_size)
 
    def forward(self, input_y, h = None):  
        samples = input_y
        rnn_out, last_rnn_hidden = self.rnn(samples, h)
        output = self.hidden2output(rnn_out.view(-1, self.hidden_dim))
        return output.view(samples.shape[0],samples.shape[1],self.output_size), last_rnn_hidden

In [16]:
### " F-norm of A
class L2LossFun(nn.Module):
    
    def __init__(self):
        super(L2LossFun, self).__init__()
    def forward(self, A_Est, A_True):
        gap = math.sqrt(np.sum((A_Est - A_True)**2))
        return gap

class LinfLossFun(nn.Module):
    
    def __init__(self):
        super(LinfLossFun, self).__init__()
    def forward(self, A_Est, A_True):
        gap = np.max(np.abs(A_Est-A_True))
        return gap

In [17]:
dfRaw = pd.read_csv("macro40.csv") 
dfP = dfRaw.drop("Date",axis=1) # only in the first time
df = np.array(dfP)
### "Data preprocessing
scaler = MinMaxScaler(feature_range=(0, 1))
df = scaler.fit_transform(df)

In [18]:
# Hyper Parameters
N = 40
output_size = 40
Smp_size = 100
q = 1 #hidden dimension
Pred_size = len(df)-Smp_size
# the layer is set to be 1 and fixed

In [19]:
names = {}
names["predErrorRNN1"] =  []
names["predErrorRNN1Linf"] =  []
names["LTRRNN1_pred"] = []

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.parameter import Parameter
import copy
from copy import deepcopy
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
 
# initialization
p , q, output_size, K = (N, 1, N, 100)

# transform the time series involving
def create_dataset(y, look_back=1):
    dataYp,dataYc = [], [] #Ypast, Ycurrent
    for i in range(len(y)-look_back):
        dataYp.append(y[i,:])
        dataYc.append(y[i+look_back, :])
    return np.array(dataYp), np.array(dataYc)
 
# transform data to tensor in torch
def to_torch(state):
    state = torch.from_numpy(state).float()
    return state

In [25]:
distance = L2LossFun()
distanceLinf = LinfLossFun()

t1_start = perf_counter() 

for k in range(Pred_size):
    y = df[(Smp_size+k-100):(Smp_size+k+1),:]
    inY, outY = create_dataset(y=y, look_back=1) #Fixed size at 100!
    # split into train and test sets
    train_size = 100
    test_size = 1
    trainInY,trainOutY = inY[0:train_size-1], outY[0:train_size-1]
    testInY,testOutY =  inY[train_size-1:], outY[train_size-1:]
    trainInY = np.reshape(trainInY, (trainInY.shape[0], 1, p))
    trainOutY = np.reshape(trainOutY, (trainOutY.shape[0], 1, p))
    testInY = np.reshape(testInY, (testInY.shape[0], 1, p))
    testOutY = np.reshape(testOutY, (testOutY.shape[0], 1, p))

    model = RNN(p = N,q = 1, output_size = output_size)
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum=0.9)

    loss_last = 1000
    loss_new = 0

    i = 0
    while abs(loss_last - loss_new)  > 0.00000001:
        if i > 0:
            loss_last = loss_new  

        model.zero_grad()
        target = torch.from_numpy(trainOutY).float()
        output,_ = model(torch.from_numpy(trainInY).float())

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        output, h = model(torch.from_numpy(trainInY).float())
        loss_new = loss.item()
        i = i + 1
        #print(loss_new)
    testPredict,_ = model(torch.from_numpy(testInY).float(), h)
    testPredict = testPredict.detach().numpy()
    # invert predictions
    testPredict_r = scaler.inverse_transform(testPredict[:,0, :])
    testY_r = scaler.inverse_transform(testOutY[:,0,:])
    names["LTRRNN1_pred"].append(testPredict_r)
    predError = distance(testPredict_r, (testY_r))
    names["predErrorRNN1"].append(predError)
    print("PredError is {}.".format(predError))
    predErrorLinf = distanceLinf(testPredict_r, testY_r)
    names["predErrorRNN1Linf"].append(predErrorLinf)
    print("PredErrorLinf is {}.".format(predErrorLinf))
    
    print(k)

t1_stop = perf_counter() 
print("Elapsed time during the whole program in seconds:", 
                                        t1_stop-t1_start) 
Real_100RNN1s = names
torch.save(Real_100RNN1s, "Real_100RNN1s.py")

PredError is 106.96855666771918.
PredErrorLinf is 81.85963656440917.
0
PredError is 85.55833945047092.
PredErrorLinf is 43.291851301347656.
1
PredError is 153.77246324830426.
PredErrorLinf is 107.0088701917285.
2
PredError is 110.10902059539248.
PredErrorLinf is 62.26904773712158.
3
PredError is 147.75246177977473.
PredErrorLinf is 107.97310664192383.
4
PredError is 120.88870381660865.
PredErrorLinf is 73.91723734871093.
5
PredError is 109.0157464332781.
PredErrorLinf is 56.9447708798999.
6
PredError is 96.97840059719695.
PredErrorLinf is 55.02781448364258.
7
PredError is 91.99736445703661.
PredErrorLinf is 71.99861996635255.
8
PredError is 110.94767248777906.
PredErrorLinf is 53.22478699684143.
9
PredError is 100.8416713146645.
PredErrorLinf is 59.281800909196775.
10
PredError is 102.41508941064268.
PredErrorLinf is 51.14769134521484.
11
PredError is 154.5092000436391.
PredErrorLinf is 116.57266362174805.
12
PredError is 181.24157370766238.
PredErrorLinf is 84.0798496246338.
13
PredEr